Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "John Ortiz"
COLLABORATORS = ""

---

<a id='top'></a>

# CSCI 3202, S21 Assignment 6

Shortcuts:  [top](#top) - [1](#p1) | [1a](#p1a) | [1b](#p1b) | [1c](#p1c) | [1d](#p1d) - [2](#p2) | [2a](#p2a) | [2b](#p2b) | [2c](#p2c) | [2d](#p2d) | [2e](#p2e)

# Assignment Overview

This assignment will ask you to implement a Bayesian Network. Problem 1 focuses on modeling heart disease, while Problem 2 models a decision making process.

Here's a summary of the tasks required and the associated points:

| Problem #  | Tasks                                                  | Points  |
|:---        |:---                                                    |:---:    |
| 1a         | Code: Complete implementation of `BayesNet` class      | 9       |
| 1b         | Code: Implement `get_prob` function                    |  5      |
| 1c         | Written: Calculate `P(HBP)` by hand                    | 10      |
| 1d         | Written: Explain how evidence changes probabilities    |  6      |
| 2a         | Code: Modify `P` to handle more complex variables      |  5      |
| 2b         | Code: Modify `get_prob` function                       |   5     |
| 2c         | Code: Calculate inferences                             |  5      |
| 2d         | Written: Calculate inferences by hand                  |  8      |
| 2e         | Code: Use approximate Bayesian calculation             |  7      |
| Total      |                                                        | 60      |

In [2]:
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<a id='p1'></a>[Back to top](#top)

# Problem 1:  Bayesian network to model heart disease

The following Bayesian network is based loosely on a study that examined heart disease risk factors in 167 elderly individuals in South Carolina. Note that this figure uses Y and N to represent Yes and No, whereas in class we used the equivalent T and F to represent True and False Boolean values.

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/hw05_bayesnet_heartdisease.png" style="width: 650px;"/>

<a id='p1a'></a>
[Back to top](#top)

## (1a) - 9 points

Task: create a `BayesNet` object to model this problem. We have provided code for the (conditional) probability `P` function and `BayesNode` class. 

You can code the `BayesNet` class however you want, subject to the following constraints:
1. the nodes are represented using the `BayesNode` class and can work with the `P` function for probabilities,
1. your `BayesNet` structure keeps track of which nodes are in the Bayes net, as well as
1. which nodes are the parents/children of which other nodes.

Some *suggested* skeleton codes for a class structure are given. The suggestions for methods to implement are in view of the fact that we will need to calculate some probabilities, which is going to require us to `find_node`s and `find_values` that nodes can take on.

In [3]:
# These are provided for you, no need to edit.

## For the sake of brevity...
T, F = True, False

## Define probability node takes on value given evidence
def P(node, value, evidence={}):
    '''The probability distribution for P(var | evidence), 
    when all parent variables are known (in evidence)'''
    if len(node.parents)==1:
        # only one parent
        row = evidence[node.parents[0]]
    else:
        # multiple parents
        row = tuple(evidence[parent] for parent in node.parents)
    return node.cpt[row] if value else 1-node.cpt[row]

## Define BayesNode object
class BayesNode:
    
    def __init__(self, name, parents, values, cpt):
        if isinstance(parents, str):
            parents = parents.split()
            
        if len(parents)==0:
            # if no parents, empty dict key for cpt
            cpt = {(): cpt}
        elif isinstance(cpt, dict):
            # if there is only one parent, only one tuple argument
            if cpt and isinstance(list(cpt.keys())[0], bool):
                cpt = {(v): p for v, p in cpt.items()}

        self.variable = name
        self.parents = parents
        self.cpt = cpt
        self.values = values
        self.children = []
        
    def __repr__(self):
        return repr((self.variable, ' '.join(self.parents)))    

Your solution code to implement the `BayesNet` class goes below. Note that you can structure the internal workings as you wish, but the constructor, `find_node`, and `find_values` must match what is used in the tests.

In [4]:
##===============================================##
## Suggested skeleton codes for a BayesNet class ##
##===============================================##

class BayesNet:
    '''Bayesian network containing only boolean-variable nodes.'''

    def __init__(self, nodes):
        '''Initialize the Bayes net by:
        1. starting empty lists to store future added nodes and variable names
        2. storing nodes passed to the contstructor. The passed list 
            should be BayesNode class objects ordered from parents to 
            children (`top` to `bottom`, from causes to effects)
        Estimated length ~ 5 lines of code
        '''
        self.NodeList= [x for x in nodes ]
        self.Variables = [x.variable for x in nodes ]
        self.AddedNodes =[]
        #self.Truth=True#used for current variable in calling the p function    
    def find_node(self, var):
        '''Find and return the BayesNode in the net with name `var`
        Return an exception if the requested variable does not exist.
        Estimated length ~ 5 lines of code
        '''
        
            
        for x in self.NodeList:
            if (x.variable==var):
                return x
        raise LookupError(var+"not in Bayes Net") 
        
    def add(self, node):
        '''This function adds a new BayesNode to the BayesNet. 
        The starter code checks that the parents should all already 
        be in the net, and the variable itself should not be.
        Add node and its variables to the appropriate lists and update parent nodes
        Estimated length ~ 5 lines of code
        '''
        assert node.variable not in self.variables
        assert all((parent in self.variables) for parent in node.parents)
        
        #add node
        self.NodeList.append(node)
        self.variables.append(node.variable)
        self.AddedNodes.append(node.variable)
        


        
    def find_values(self, var):
        '''Return the set of possible values associated with 
        the variable `var`
        Estimated length ~ < 5 lines of code
        '''
        assert var in self.Variables
        node = self.find_node(var)
        return node.values

    
    # Do not need to modify the below function
    def __repr__(self):
        return 'BayesNet({})'.format(self.nodes)

### (1a) Tests

In [5]:
# Test setting up a simple problem

## BEGIN TESTS
parent1 = BayesNode('p1', '', [T,F], 0.3)
parent2 = BayesNode('p2', '', [T,F], 0.6)
child   = BayesNode('c', ['p1', 'p2'], [T,F], {(T,T):0.1, (T,F):0.2, (F,T):0.3, (F,F):0.4})
simpleNet = BayesNet([parent1, parent2, child])

p1T = P(parent1, T)
assert (p1T == 0.3), "parent1 should have P(true) = 0.3, your code returned %f" % p1T

cF = P(child, F, {'p1':T, 'p2':F})
assert (cF == 0.8), "child should have P(false | p1=true, p2=false) = 0.8, your code returned %f" %cF

print("All tests passed, 3 points")
## END TESTS

All tests passed, 3 points


In [6]:
## BEGIN TESTS

# Define the nodes from the problem statement
Sm = BayesNode('Sm', '', [T,F], 0.2)
ME = BayesNode('ME', '', [T,F], 0.5)
HBP = BayesNode('HBP', ['Sm', 'ME'], [T,F], {(T, T): 0.6, (T, F): 0.72, (F, T): 0.33, (F, F): 0.51})
Ath = BayesNode('Ath', '', [T,F], 0.53)
FH = BayesNode('FH', '', [T,F], 0.15)
HD = BayesNode('HD', ['Ath', 'HBP', 'FH'], [T,F], 
               {(T, T, T): 0.92, (T, T, F): 0.91, (T, F, T): 0.81, (T, F, F): 0.77,
                (F, T, T): 0.75, (F, T, F): 0.69, (F, F, T): 0.38, (F, F, F): 0.23})
Ang = BayesNode('Ang', 'HD', [T,F], {T: 0.85, F: 0.4})
Rapid = BayesNode('Rapid', 'HD', [T,F], {T: 0.99, F: 0.3})

# Create a Bayes net with those nodes and connections
bnHeart = BayesNet([Sm, ME, HBP, Ath, FH, HD, Ang, Rapid])

FHparents = FH.parents
assert (len(FHparents) == 0), "Node FH should have 0 parents, your code returned %s" % FHparents
HDparents = HD.parents
assert (len(HDparents) == 3), "Node HD should have 3 parents, your code returned %s" % HDparents

# Test find_node function
assert (bnHeart.find_node("Sm") != None), "Could not find node with name Sm"
assert (bnHeart.find_node("HD") != None), "Could not find node with name HD"
assert (bnHeart.find_node("Ang") != None), "Could not find node with name Ang"

# Test find_values function
MEvars = bnHeart.find_values("ME")
assert (MEvars == [T, F]), "Values for ME node should be [True, False], your code returned %s" % MEvars
HDvars = bnHeart.find_values("HD")
assert (HDvars == [T, F]), "Values for HD node should be [True, False], your code returned %s" % HDvars


print("All tests passed, 6 points")

## END TESTS

All tests passed, 6 points


<a id='p1b'></a>
[Back to top](#top)

## (1b) - 5 points

Implement a function `get_prob(X, e, bn)` to return the **normalized** probability distribution of variable `X` in Bayes net `bn`, given the evidence `e`.  That is, return $P(X \mid e)$. The arguments are:
* `X` is some representation of the variable you are querying the probability distribution of. Either a string (the variable name from the `BayesNode` or a `BayesNode` object itself are good options.
* `e` is some representation of the evidence your probability is conditioned on. When given an empty argument (or `None`) for `e`, `get_prob` should return the marginal distribution $P(X)$.
* `bn` is your `BayesNet` object.

You may do this using the `enumeration` algorithm (pseudocode is in the book), or by brute force (i.e., use a few `for` loops). Either way, you should be using your `BayesNet` object to keep track of all the nodes and relationships between nodes so your `get_prob` function knows these things. We have provided a few helper functions in the `PDF_discrete` class.

In [7]:

class PDF_discrete:
    '''Define a discrete probability distribution function.'''

    def __init__(self, varname='?', freqs=None):
        '''Create a dictionary of values - frequency pairs,
        then normalize the distribution to sum to 1.'''
        self.prob = {}
        self.varname = varname
        self.values = []
        if freqs:
            for (v, p) in freqs.items():
                self[v] = p
        self.normalize()

    def __getitem__(self, value):
        '''Given a value, return P[value]'''
        try:
            return self.prob[value]
        except KeyError:
            return 0

    def __setitem__(self, value, p):
        '''Set P[value] = p, input argument if '''
        if value not in self.values:
            self.values.append(value)
        self.prob[value] = p

    def normalize(self):
        '''Normalize the probability distribution and return it.
        If the sum of PDF values is 0, then return a 0
        '''

        total = sum(self.prob.values())
        if not np.isclose(total, 0.0):
            for value in self.prob:
                self.prob[value] /= total
        return self
    
def extend(s, var, val):
    """Copy the substitution s and extend it by setting var to val; return copy."""
    s2 = s.copy()
    s2[var] = val
    return s2
def enumerateAll(varList,e,bn):
    if(len(varList)<=0):
        return 1
    Y= bn.find_node(varList[0])
    restVars= varList[1:len(varList)]
    
    if Y.variable in e.keys():
        y=e[Y.variable]
        probY = P(Y,y,e)
        return probY * enumerateAll(restVars,e,bn)
    
    else:
        possible_y=Y.values#[T,F]
        ySum=0
        for y in possible_y:
            newE0 =extend(e,Y.variable,y)
            t0=P(Y,y,newE0)
            recCall = enumerateAll(restVars,newE0,bn)
            ySum += (t0*recCall)
        
        return ySum
        
def get_prob(X, e, bn):
    '''Return the conditional probability distribution of variable X
    given evidence e, from BayesNet bn. [Figure 14.9]
    Return normalized instance of PDF_discrete
    '''
    #enumeration from psuedo code from book
    pdfX =  PDF_discrete(X.variable)

    #for each value of X 
    for xi in X.values:
        
        e_xi = extend(e,X.variable,xi)
        #call to enumerate-All
        q_xi = enumerateAll(bn.Variables,e_xi,bn)
        pdfX.prob[xi] = q_xi
        

    pdfX.normalize()
    #print(T,pdfX.prob[T])
    #print(F,pdfX.prob[F])
    
    assert X.variable in e.keys()
    finalVal = e[X.variable]
   
    return pdfX.prob[finalVal]
    

### (1b) Tests

Use your `get_prob` function to calculate the following probabilities. You can print them to the screen and compare to the original Bayes net figure given to make sure this makes sense. We have given you the variable names to assign your answers to, which will be used in the test cases. *Do not* change the variable names or the tests will not pass! You *must* use the `get_prob` function to calculate these values, we will not give credit for hard-coded answers.

1. Calculate the marginal probability of `Family History` is True $P(FH=T)$ and assign to `a1`
2. Calculate the probability of *not* experiencing `Angina Pectoris`, given `Heart Disease` is observed $P(Ang=F \mid HD=T)$ and assign to `a2`
3. Calculate the probability of `High Blood Pressure`, given a person does `Smoke and/or use Alcohol` but does not get `Moderate Exercise` $P(HBP=T \mid Sm=T, ME=F)$ and assign to `a3`
4. Calculate the probability of an arbitrary individual having `Heart Disease`, $P(HD=T)$ and assign to `a4`
5. Calculate the probability that an individual does *not* have `Heart Disease`, given that `Rapid Heartbeat` was observed, $P(HD=F \mid Rapid=T)$ and assign to `a5`
6. Calculate the probability of an individual having `High Blood Pressure` if they have `Heart Disease` and a `Family History`, $P(HBP=T \mid HD=T, FH=T)$ and assign to `a6`
7. Calculate the probability that an individual is a `Smoker/Alcohol User` if they have `Heart Disease`, $P(Sm=T \mid HD=T)$ and assign to `a7`

In [8]:
# Your code to calculate the probabilities goes here
# Estimated length: 1-2 lines per calculation

X=bnHeart.find_node("FH")
e= {"FH": T}
a1= get_prob(X,e,bnHeart)

X=bnHeart.find_node("Ang")
e={'Ang':F, 'HD':T}
a2= get_prob(X,e,bnHeart)

X=bnHeart.find_node("HBP")
e={"HBP":T,'Sm':T, 'ME':F}
a3= get_prob(X,e,bnHeart)

X=bnHeart.find_node("HD")
e={"HD":T}
a4= get_prob(X,e,bnHeart)

X=bnHeart.find_node("HD")
e={"HD":F,"Rapid":T}
a5= get_prob(X,e,bnHeart)

X=bnHeart.find_node("HBP")
e={"HBP":T,"HD":T,"FH":T}
a6= get_prob(X,e,bnHeart)

X=bnHeart.find_node("Sm")
e={"Sm":T,"HD":T}
a7= get_prob(X,e,bnHeart)

In [9]:
## BEGIN TESTS
assert (np.isclose(a1, 0.15, atol=0.002)), "P(FH) = 0.15 and your code returned %f" % a1
assert (np.isclose(a2, 0.15, atol=0.002)), "P(!Ang|HD) = 0.85 and your code returned %f" % a2
assert (np.isclose(a3, 0.72, atol=0.002)), "P(HBP|Sm, ME) = 0.72 and your code returned %f" % a3
assert (np.isclose(a4, 0.661, atol=0.002)), "P(HD) = 0.662 and your code returned %f" % a4
assert (np.isclose(a5, 0.134, atol = 0.002)), "P(HD|Rapid) = 0.865 and your code returned %f" % a5
assert (np.isclose(a6, 0.548, atol = 0.002)), "P(HBP|HD,FH) = 0.548 and your code returned %f" % a6
assert (np.isclose(a7, 0.216, atol = 0.002)), "P(Sm|HD) = 0.216 and your code returned %f" % a7

print("All tests passed, 5 points")
## END TESTS

All tests passed, 5 points


<a id='p1c'></a>
[Back to top](#top)

## (1c) - 10 points

Calculate the probability of observing someone with `High Blood Pressure`, $P(HBP=T)$, *by hand*, showing all work in Markdown/LateX below.

X = HBP\
E = no evidence\
Hidden vars ={SM,ME,HD,ATH,FH,ANG,Rapid}\
P(HBP) = $\sum_{SM,ME,HD,ATH,FH,ANG,Rapid} $P(HBP | SM, ME) P(SM) P(ME) P(HD | HBP,ATH,FH) P(ATH) P(FH) P(Ang | HD) P(Rapid | HD)
- which can be reduced with summing out hidden variables that are not dependent on x or E or their parents

P(HBP) = $\sum_{SM,ME} $P(HBP | SM, ME) P(SM) P(ME)$\sum_{HD,ATH,FH}$ P(HD | HBP,ATH,FH) P(ATH) P(FH) $\sum_{Ang}$ P(Ang | HD)$\sum_{Rapid}$ P(Rapid | HD)


| SM | ME | $\texttt{P(HBP | SM, ME) P(SM) P(ME)}$ |
|-----|-------|------|
|T |T|(.60)(.20)(.50)=.06|
|T |F|(.72)(.20)(.50)=.072|
|F |T|(.33)(.80)(.50)=.132|
|F |F|(.51)(.80)(.50)=.204|

Than we sum each rows answers $.06+.072+.132+.204= .468

### (1c) Tests
**Verify** your calculation using your `get_prob` function. Store your answer in the provided `pHBP` variable.

In [10]:
X=bnHeart.find_node("HBP")
e={"HBP":T}
pHBP= get_prob(X,e,bnHeart)

In [11]:
## BEGIN TESTS
assert (np.isclose(pHBP, 0.468, atol=0.002)), "P(HPB) = 0.468, your code returned %f" % pHBP2

print("All tests passed, 2 points")
## END TESTS

All tests passed, 2 points


<a id='p1d'></a>
[Back to top](#top)

## (1d) - 6 points

How would you expect the probability in `a7` from problem (1b) to change if you also know the individual has `High Blood Pressure`?  Verify your hypothesis by calculating the relevant probability.

I would expect the probability of smoking and alocohol would increase as smoking and alcohol is a parent of high blood pressure. If we look at P(HBP | SM) each time SM is true the probability of have HBP is higher (.60,.72) as well. So in the enumerateAll function our sums of products of conditional probibilities will also be higher as a any number multiplies by a higher decimal will be a bigger product than ones multiplied than smaller decimals. So with the extra evidence we would not include P(!HBP| SM) as before we would have had to calculate P(HBP| SM) and P(!HBP | SM) in the enumerate all function.  

In [12]:
# Your code to verify here (not graded)

X=bnHeart.find_node("Sm")
e={"Sm":T,"HD":T,"HBP":T}
a7_v2= get_prob(X,e,bnHeart)
print("Orginal Probability of A7: "+str(a7)+" VS with HBP being True: "+str(a7_v2))

Orginal Probability of A7: 0.2163440784303391 VS with HBP being True: 0.28205128205128205


How would you expect the probability in the last part to change if you also know that the individual does *not* get `Moderate Exercise` (in addition to having `Heart Disease` and `High Blood Pressure`)?  Explain your answer using concepts from class.  Verify your answer by calculating the relevant probability.

The probability from last would decrease if we know the individual does not get Moderate Exercise. As the difference this would makes is when calculating HBP we would only include the P(HBP| Sm: T, ME:F) which eliminates all the options in  P(HBP| SM,ME) to just one being .72 assuming HBP is true from last part. So when doing total summations we will not add the probability product of (P(HBP| SM, ME: F) * P(ME: F) * rest) as it will just get skipped. Hence I think it should only be a small decrease.

In [13]:
# Your code to verify here (not graded)

X=bnHeart.find_node("Sm")
e={"Sm":T,"HD":T,"HBP":T,"ME":F}
a7_v3= get_prob(X,e,bnHeart)
print("Orginal Probability of A7_v2: "+str(a7_v2)+" VS with ME being False: "+str(a7_v3))

Orginal Probability of A7_v2: 0.28205128205128205 VS with ME being False: 0.2608695652173913


<a id='p2'></a>
[Back to top](#top)

<img src="https://inhabitat.com/wp-content/blogs.dir/1/files/2014/02/norman-bike-riding-dog.png" style="width: 350px;"/>

# Problem 2:  Bayesian network to model decision-making

Let's consider using a Bayesian network to model our decision about whether or not to ride our bike to work today.  This decision depends heavily on the weather, so let's focus on that.

In class, we focused on Boolean variables.  For example, we might base our biking decision on whether or not it is raining.  But in reality, it probably matters *how hard* it is raining.  So suppose we break the variable `Rain` up into three discrete bins: `none`, `light` and `heavy`.

The temperature also factors into our decision.  There is definitely a sweet spot, where temperatures are neither too warm nor too cold, so it is very likely we would enjoy riding our bike.  So we can model the variable `Temperature` also using three discrete bins: `cold`, `moderate` and `warm`.

So a Bayesian network to model our decision for whether or not to bike to work could be as follows, where the first letter of each discrete bin is used to denote that variable value (i.e., `R=h` stands for heavy rain conditions).

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/bayesnet_biking2.png" style="width: 650px;"/>

<a id='p2a'></a>
[Back to top](#top)

## (2a) - 5 points

Modify the `P` probability function to be able to handle these ternary parent nodes.

In [14]:
# modified function for conditional probabilities,
# to handle ternary (or more) case
def P(var, value, evidence={}):
    '''The probability distribution for P(var | evidence), 
    when all parent variables are known (in evidence)
    Consider how the calculation differs based on the number of parents.
    Estimated length: ~10 lines of code
    '''
    '''The probability distribution for P(var | evidence), 
    when all parent variables are known (in evidence)'''
    if len(var.parents)==1:
        #print("1 parent")
        # only one parent
        row = evidence[node.parents[0]]
    else:
        # multiple parents
        #print("multi parent")
        row = tuple(evidence[parent] for parent in var.parents)
    #if (len(row) <=0):
    if(type(value)!= bool):
        if value in var.cpt[row].keys():
            return var.cpt[row][value]
        else:
            
            sumVals = sum(var.cpt[row].values())
            #print(round(sumVals,2))
            return round(1-sumVals,2)
    else:
        return var.cpt[row] if value else 1-var.cpt[row]
        #return var.cpt[row]

Below we'll set up `BayesNode` objects for each of `Rain`, `Temp` and `Bike`, and create a `BayesNet` object to model the Bayesian network for this decision.

In [15]:
# Set up the Bayes net
n,l,h,c,m,w,T,F = 'None','Light','Heavy','Cold','Moderate','Warm',True,False

rain = BayesNode('Rain', '', [n,l,h], {n : 0.8, l : 0.15})
temp = BayesNode('Temp', '', [c,m,w], {c : 0.3, m : 0.6 })
bike = BayesNode('Bike', ['Rain', 'Temp'], [T, F], {(n,c) : 0.7, (n,m) : 0.99, (n,w) : 0.9,
                                                    (l,c) : 0.4, (l,m) : 0.6 , (l,w) : 0.5,
                                                    (h,c) : 0.2, (h,m) : 0.4 , (h,w) : 0.3})
bnRide = BayesNet([rain, temp, bike])

**Verify** that your modified probability function `P` is working by making the following calculations. You may print the output to screen and compare to what you expect from the figure above.

1. Calculate the marginal probability of no rain is $P(Rain=n)=0.8$ and store as `p2a1`
1. Calculate the marginal probability of light rain is $P(Rain=l)=0.15$ and store as `p2a2`
1. Calculate the marginal probability of heavy rain is $P(Rain=h)=0.05$ and store as `p2a3`
1. Calculate the probability of biking given that it is raining heavily and the temperature is cold, is $P(Bike=T \mid Rain=h, Temp=c)=0.2$ and store as `p2a4`

In [16]:
# Your code to calculate goes here:

p2a1 = None
X=bnRide.find_node("Rain")
p2a1 = P(X,n)

p2a2 = None
X=bnRide.find_node("Rain")
p2a2 = P(X,l)

p2a3 = None
X=bnRide.find_node("Rain")
p2a3 = P(X,h)


p2a4 = None
X=bnRide.find_node("Bike")
e= {"Rain":h,"Temp": c}
p2a4 = P(X,T,e)


### (2a) Tests

In [17]:
## BEGIN TESTS
assert (np.isclose(p2a1, 0.8, atol = 0.002)), "P(R=n) = 0.8, your code returned %f" % p2a1
assert (np.isclose(p2a2, 0.15, atol = 0.002)), "P(R=l) = 0.15, your code returned %f" % p2a2
assert (np.isclose(p2a3, 0.05, atol = 0.002)), "P(R=h) = 0.05, your code returned %f" % p2a3
assert (np.isclose(p2a4, 0.2, atol = 0.002)), "P(B|R=h, T=c) = 0.2, your code returned %f" % p2a4

print("Tests passed, 5 points")
## END TESTS

Tests passed, 5 points


<a id='p2b'></a>
[Back to top](#top)

## (2b) - 5 points

Make any necessary modifications to your `get_prob` function from Problem 1, so that you can use it to calculate marginal probabilities and conditional probabilities for this problem. It is possible that your function does not require any modifications.

In [18]:
#did not need to change anything

1. Use `get_prob` to calculate $P(Bike)$, the probability distribution for whether or not you will ride your bike on any given day. Store the value of $P(B=True)$ in the variable `p2a5`.
2. Use `get_prob` to calculate the probability that you will ride your bike, given that it is lightly raining and store the value in the variable `p2a6`.

In [19]:
# Your code here to calculate

p2a5 = None
X=bnRide.find_node("Bike")
e={"Bike":T}
p2a5= get_prob(X,e,bnRide)

p2a6 = None
X=bnRide.find_node("Bike")
e={"Bike":T,"Rain":l}
p2a6= get_prob(X,e,bnRide)

### (2b) Tests

In [20]:
## BEGIN TESTS
assert (np.isclose(p2a5, 0.811, atol=0.002)), "P(Bike) = 0.811, your code returned %f" % p2a5
assert (np.isclose(p2a6, 0.529, atol=0.002)), "P(B=T|R=l) = 0.529, your code returned %f" %p2a6

print("All tests passed, 5 points")
## END TESTS

All tests passed, 5 points


<a id='p2c'></a>
[Back to top](#top)

## (2c) - 5 points

We are trapped indoors because some jerk gave us a ton of Intro to Artificial Intelligence homework to do.  Suppose we look out the window and see people biking. They sure do look like they're having fun! *Given* this information, we can actually make inferences regarding the temperature outside!  What is the probability distribution for temperature, given that we observe people biking?

First, compute this using your `get_prob` function. Store the probability for each temperature in the corresponding variable of `p2cold`, `p2moderate`, and `p2warm`, respectively.

In [21]:
# Your solution goes here

p2cold = get_prob(bnRide.find_node("Temp"),{"Temp":c,"Bike":T},bnRide)
p2moderate = get_prob(bnRide.find_node("Temp"),{"Temp":m,"Bike":T},bnRide)
p2warm = get_prob(bnRide.find_node("Temp"),{"Temp":w,"Bike":T},bnRide)



### (2c) Tests

In [22]:
## BEGIN TESTS
assert (np.isclose(p2cold, 0.233, atol=0.002)), "P(T=c|B=True) = 0.233, your code returned %f" % p2cold
assert (np.isclose(p2moderate, 0.667, atol=0.002)), "P(T=m|B=True) = 0.667, your code returned %f" % p2moderate
assert (np.isclose(p2warm, 0.099, atol=0.002)), "P(T=w|B=True) = 0.099, your code returned %f" % p2warm

print("Tests passed, 5 points")
## END TESTS

Tests passed, 5 points


<a id='p2d'></a>
[Back to top](#top)

## (2d) - 8 points

Confirm your answer to **2c** by hand, showing *all* relevant work below in a LateX/Markdown cell.

X = Temp\
E = {"Bike": T}\
Hidden vars ={Rain}\
P(Temp) = $\alpha_{Temp}\sum_{Rain} $P(Bike:T | Temp, Rain) P(Temp) P(Rain) 



| Temp | Rain | $\texttt{P(Temp) P(Rain) P(Bike:T | Temp, Rain) }$ |
|-----|-------|------|
|c |n|(.30)(.8)(.7)=.168|
|c |l|(.30)(.15)(.4)=.018|
|c |h|(.30)(.05)(.2)=.003|
|m |n|(.60)(.8)(.99)=.4752|
|m |l|(.60)(.15)(.6)=.054|
|m |h|(.60)(.05)(.4)=.012|
|w |n|(.10)(.8)(.9)=.072|
|w |l|(.10)(.15)(.5)=.0075|
|w |h|(.10)(.05)(.3)=.0015|

Than we sum each rows pertaining the temperatures state we sum for c,m,w:
   - P(Temp: c) $= .168+.018+.003= .189$
   - P(Temp: m) $= .4752+.054+.012= .5412$
   - P(Temp: w) $= .072+.0075+.0015= .081$
   
   
However we have to normalize since they do not add up to 1:
   - $total= \sum_{i=c,m,w} P(Temp_i | Bike: T) = .189+.5412+.081=.8112$
   - P(Temp: c) $=\frac{.189}{total}=\frac{.189}{.8112}=.23299 $
   - P(Temp: m) $=\frac{.5412}{total}=\frac{.5412}{.8112}=.66716$
   - P(Temp: w) $=\frac{.081}{total}=\frac{.081}{.8112}=.09985$
   
Which match 2c answers

<a id='p2e'></a>
[Back to top](#top)

## (2e) - 7 points

Finally, confirm your confirmation of the probability distribution for `Temp` by using approximate Bayesian computation and 10,000 samples.  That is, use the **prior sampling** and **"rejection sampling"** techniques from class to estimate the probabilities associated with each possible value for `Temp`, given that there are people biking outside. 

Repeat this calculation at least 10 times and display the results in a way that makes sense for the reader. Write a few sentences about your observations.

In [23]:
# TODO:
# - Sample from priors
# - Sample from distribution of Bike, given the parents
# - Estimate P(T=c|B=True)
# - Estimate P(T=m|B=True)
# - Estimate P(T=w|B=True)
# - Repeat above and record results

# YOUR CODE HERE
#prior sampling
def priorSamps():
    #for rain
    rainChoice= np.random.choice([n,l,h],p=[.8,.15,.05])
    #for temp
    tempChoice= np.random.choice([c,m,w],p=[.3,.6,.1])
    return rainChoice,tempChoice
def bikeSamp():
    r_C,t_C = priorSamps()
    X=bnRide.find_node("Bike")
    bikeProb= X.cpt[(r_C,t_C)] #cpt only used for condition probability table to lazy to write
    b_C= np.random.choice([T,F],p=[bikeProb,1-bikeProb])
    return r_C,t_C,b_C

def runSim(n,verbose=False):
    countTemp_c =0
    countTemp_m =0
    countTemp_w =0
    countBike = 0 #for how many trues 
    #est_c=countTemp_c/n
    #est_m=countTemp_m/n
    #est_w=countTemp_w/n
    for i in range(n):
        r_C,t_C,b_C=bikeSamp()
        #while(b_C== F): #dont care when bike is False
        #    r_C,t_C,b_C=bikeSamp()
        if(t_C == c and b_C == T):
            countTemp_c +=1
            countBike+=1
        elif(t_C == m and b_C == T):
            countTemp_m +=1
            countBike+=1
        elif(t_C == w and b_C == T):
            countTemp_w +=1
            countBike+=1
        #update each one as n increases
        est_c=countTemp_c/countBike
        est_m=countTemp_m/countBike
        est_w=countTemp_w/countBike
        if(verbose):
            if((i+1)%100==0): #+1 for index iterations from 1 
                print("Iteration "+str(i+1)+": ")
                print("Estimate P(T=c|B=True): "+str(est_c))
                print("Estimate P(T=m|B=True): "+str(est_m))
                print("Estimate P(T=w|B=True): "+str(est_w))
    assert(countTemp_c+countTemp_m+countTemp_w==countBike)
    print("(Final) Iteration "+str(n)+": ")
    print("Estimate P(T=c|B=True): "+str(est_c))
    print("Estimate P(T=m|B=True): "+str(est_m))
    print("Estimate P(T=w|B=True): "+str(est_w))
    return countBike/n #get just estimate of riding bike for unit test
p2e = runSim(10000)

(Final) Iteration 10000: 
Estimate P(T=c|B=True): 0.23257818271361733
Estimate P(T=m|B=True): 0.6673233193794632
Estimate P(T=w|B=True): 0.10009849790691948


Your observations go below. How do these compare to your calculation in the last part? How do the estimates change between repetitions?

These samples seem to be pretty close to the calculations in the last part. I knew from my data science class a descent amount about simulation so I knew I should have a descently sized n. As n increase the estimates climb closer to their true value. I left a parameter in my runSim function(verbose) if set to true in that prints the estimates every 100th sample to demonstrate this.

As a "Unit Test", check what the probability of riding your bike is, given no other information.  Make sure this approximately matches your answer to **2b**.

In [24]:
# Solution (not graded):

print("From above simulation probability of riding bike with no other information is "+str(p2e))
assert (np.isclose(p2e, 0.811, atol=0.05)), "P(Bike) = 0.811, your code returned %f" % p2a5


From above simulation probability of riding bike with no other information is 0.8122


[Back to top](#top)